In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, explode, struct, collect_list, element_at, first, row_number, concat, from_json, udf
from pyspark.sql.types import FloatType, StructType, StructField, StringType, IntegerType, MapType, ArrayType
import requests
import json

In [2]:
spark = SparkSession.builder \
    .appName("mini-project") \
    .getOrCreate()

In [120]:
api = "https://www.pegelonline.wsv.de/webservices/rest-api/v2/stations.json?includeTimeseries=true&hasTimeseries=WV&includeForecastTimeseries=true"
responses = requests.get(api).json()
print(responses[0])

{'uuid': 'e6d68ab7-5c27-4f25-896f-11dbf04056cd', 'number': '10089006', 'shortname': 'VILSHOFEN', 'longname': 'VILSHOFEN', 'km': 2249.5, 'agency': 'STANDORT REGENSBURG', 'longitude': 13.182352727075328, 'latitude': 48.63719446076585, 'water': {'shortname': 'DONAU', 'longname': 'DONAU'}, 'timeseries': [{'shortname': 'W', 'longname': 'WASSERSTAND ROHDATEN', 'unit': 'cm', 'equidistance': 15, 'gaugeZero': {'unit': 'm. ü. NHN', 'value': 297.043, 'validFrom': '2019-01-01'}}, {'shortname': 'WV', 'longname': 'WASSERSTANDVORHERSAGE', 'unit': 'cm', 'equidistance': 120, 'start': '2024-07-18T19:00:00+02:00', 'end': '2024-07-22T05:00:00+02:00', 'comment': {'shortDescription': 'nwv-bfg', 'longDescription': 'Vorhersagen und Abschätzungen vom: 18.07.2024 um 05:00 Uhr, Quelle: Bundesanstalt für GewässerkundeWeitere Informationen zur Unterscheidung von Vorhersage und Abschätzung finden Sie auf den Seiten der Bundesanstalt für Gewässerkunde'}}, {'shortname': 'LT', 'longname': 'LUFTTEMPERATUR', 'unit': '°C

In [121]:
timeseries_data = [(response['uuid'], response['uuid']+str(item_id), json.dumps(item)) 
                   for response in responses
                   for item_id, item in enumerate(response['timeseries'])]
[item.pop('timeseries') for item in responses]

[[{'shortname': 'W',
   'longname': 'WASSERSTAND ROHDATEN',
   'unit': 'cm',
   'equidistance': 15,
   'gaugeZero': {'unit': 'm. ü. NHN',
    'value': 297.043,
    'validFrom': '2019-01-01'}},
  {'shortname': 'WV',
   'longname': 'WASSERSTANDVORHERSAGE',
   'unit': 'cm',
   'equidistance': 120,
   'start': '2024-07-18T19:00:00+02:00',
   'end': '2024-07-22T05:00:00+02:00',
   'comment': {'shortDescription': 'nwv-bfg',
    'longDescription': 'Vorhersagen und Abschätzungen vom: 18.07.2024 um 05:00 Uhr, Quelle: Bundesanstalt für GewässerkundeWeitere Informationen zur Unterscheidung von Vorhersage und Abschätzung finden Sie auf den Seiten der Bundesanstalt für Gewässerkunde'}},
  {'shortname': 'LT',
   'longname': 'LUFTTEMPERATUR',
   'unit': '°C',
   'equidistance': 60},
  {'shortname': 'WT',
   'longname': 'WASSERTEMPERATUR',
   'unit': '°C',
   'equidistance': 60,
   'comment': {'shortDescription': 'Wassertemperatursonde ausgefallen.',
    'longDescription': 'Wassertemperatursonde ausge

In [136]:
response_str = json.dumps(responses)

# Load the JSON string into a Spark DataFrame
response_df = spark.read.json(spark.sparkContext.parallelize([response_str]))

# Flatten the dataframe
response_df = response_df.select(
    col("agency"),
    col("km").cast("double"),
    col("latitude").cast("double"),
    col("longitude").cast("double"),
    col("longname"),
    col("number"),
    col("shortname"),
    col("uuid"),
    col("water.longname").alias("water_longname"),
    col("water.shortname").alias("water_shortname")
)

In [137]:
response_df.printSchema()
response_df.show()

root
 |-- agency: string (nullable = true)
 |-- km: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- longname: string (nullable = true)
 |-- number: string (nullable = true)
 |-- shortname: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- water_longname: string (nullable = true)
 |-- water_shortname: string (nullable = true)

+-------------------+------+------------------+------------------+--------------------+--------+--------------------+--------------------+--------------+---------------+
|             agency|    km|          latitude|         longitude|            longname|  number|           shortname|                uuid|water_longname|water_shortname|
+-------------------+------+------------------+------------------+--------------------+--------+--------------------+--------------------+--------------+---------------+
|STANDORT REGENSBURG|2249.5| 48.63719446076585|13.182352727075328|           VILSHO

In [124]:
"""
    Apply AVE model, with four tables:
    - "response": contains the information that every record has
    - "Timeseries Item": each row corresponds to an item of timeseries (each timeseries field in a record can have multiple items)
    - "Timeseries Attribute": the field appeared in a timeseries item. We might want to track its type (?) and name
    - "Timeseries Attribute Value": contains the value, the attribute (that the value corresponds to), and the item which the mentioned attribute belongs to

    Difficulty:
    - How can we make sure two fields are different/the same?
    - The value always has to be stored as string/text
    - Not easy to impute the type of a field 
"""

def parse_key_value(json_dict, parent):
    key_value_list = []
    for key, value in json_dict.items():
        if isinstance(value, dict):
            if parent == "": next_parent = key
            else: next_parent = parent + "." + key
            key_value_list.extend(parse_key_value(value, next_parent))
        else:
            if parent == "": this_key = key
            else: this_key = parent + "." + key
            key_value_list.append((this_key, str(value)))
    return key_value_list

def read_key_value(json_str):
    json_dict = json.loads(json_str.replace("\'", "\""))
    print(json_dict)
    key_value_list = parse_key_value(json_dict, "")
    return key_value_list
    
timeseries_df = spark.createDataFrame(timeseries_data, ['uuid', 'item_id', 'timeseries_data'])

schema = ArrayType(StructType([
    StructField("key", StringType(), True),
    StructField("value", StringType(), True)
]))
explode_udf = udf(read_key_value, schema)

timeseries_parsed_df = timeseries_df.withColumn("timeseries_parsed", explode_udf(col('timeseries_data')))
timeseries_parsed_df.show(1,truncate=False)

+------------------------------------+-------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|uuid                                |item_id                              |timeseries_data                                                                                                                                                               |timeseries_parsed                                                                                                                                                             |
+------------------------------------+-------------------------------------+----------------------------------------------------------------------

In [125]:
timeseries_parsed_df = timeseries_parsed_df.withColumn("exploded_pair", explode(col('timeseries_parsed')))
timeseries_parsed_df = timeseries_parsed_df.withColumn("key", col("exploded_pair").getItem("key")) \
                .withColumn("value", col("exploded_pair").getItem("value"))
timeseries_parsed_df = timeseries_parsed_df.drop("timeseries_data", "exploded_pair","timeseries_parsed")
timeseries_parsed_df.show(truncate=False)

+------------------------------------+-------------------------------------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|uuid                                |item_id                              |key                     |value                                                                                                                                                                                                                                   |
+------------------------------------+-------------------------------------+------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [129]:
timeseries_item = timeseries_df.select(col("item_id"),col("uuid"))
timeseries_item.show(truncate=False)

+-------------------------------------+------------------------------------+
|item_id                              |uuid                                |
+-------------------------------------+------------------------------------+
|e6d68ab7-5c27-4f25-896f-11dbf04056cd0|e6d68ab7-5c27-4f25-896f-11dbf04056cd|
|e6d68ab7-5c27-4f25-896f-11dbf04056cd1|e6d68ab7-5c27-4f25-896f-11dbf04056cd|
|e6d68ab7-5c27-4f25-896f-11dbf04056cd2|e6d68ab7-5c27-4f25-896f-11dbf04056cd|
|e6d68ab7-5c27-4f25-896f-11dbf04056cd3|e6d68ab7-5c27-4f25-896f-11dbf04056cd|
|616dd98e-816d-4e17-b8cd-57b163dbc8a20|616dd98e-816d-4e17-b8cd-57b163dbc8a2|
|616dd98e-816d-4e17-b8cd-57b163dbc8a21|616dd98e-816d-4e17-b8cd-57b163dbc8a2|
|616dd98e-816d-4e17-b8cd-57b163dbc8a22|616dd98e-816d-4e17-b8cd-57b163dbc8a2|
|616dd98e-816d-4e17-b8cd-57b163dbc8a23|616dd98e-816d-4e17-b8cd-57b163dbc8a2|
|616dd98e-816d-4e17-b8cd-57b163dbc8a24|616dd98e-816d-4e17-b8cd-57b163dbc8a2|
|53d40547-8a09-4b25-988c-2e6d8d8d98ee0|53d40547-8a09-4b25-988c-2e6d8d8d98ee|

In [133]:
timeseries_attribute_df = timeseries_parsed_df.select(col("key").alias("attribute")).dropDuplicates()
timeseries_attribute_df.show(truncate=False)

+------------------------+
|attribute               |
+------------------------+
|equidistance            |
|longname                |
|comment.shortDescription|
|end                     |
|gaugeZero.unit          |
|unit                    |
|comment.longDescription |
|gaugeZero.validFrom     |
|start                   |
|shortname               |
|gaugeZero.value         |
+------------------------+



In [135]:
timeseries_attr_val_df = timeseries_parsed_df.select(col("item_id"), col("key").alias("attribute"), col("value"))
timeseries_attr_val_df.show(truncate=False)

+-------------------------------------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|item_id                              |attribute               |value                                                                                                                                                                                                                                   |
+-------------------------------------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|e6d68ab7-5c27-4f25-896f-11dbf04056cd0|shortname               |W                                         

In [138]:
import psycopg2
from psycopg2.extras import execute_values

def insert_data(table, df, conn_param):
    columns = [f'"{col}"' for col in df.schema.names]
    data = [tuple(row) for row in df.collect()]

    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    insert_query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES %s"
    execute_values(cursor, insert_query, data)
    conn.commit()
    cursor.close()
    conn.close()


In [139]:
conn_params = {
    "dbname": "db",
    "user": "postgres",
    "password": "12345",
    "host": "pg_container",
    "port": 5432
}
insert_data("responses", response_df, conn_params)
insert_data("timeseries_item", timeseries_item, conn_params)
insert_data("timeseries_attribute", timeseries_attribute_df, conn_params)
insert_data("timeseries_value", timeseries_attr_val_df, conn_params)



UndefinedColumn: column "item_id" of relation "timeseries_value" does not exist
LINE 1: INSERT INTO timeseries_value ("item_id", "attribute", "value...
                                      ^


In [7]:
"""
    Basically, there are five types of data the timeseries contains:
    - WASSERSTAND ROHDATEN: water level raw data
    - WASSERSTANDVORHERSAGE: water level forecast
    - WASSERTEMPERATUR: water temperature
    - WASSERTEMPERATUR ROHDATEN: water temperature forecast (?)
    - LUFTTEMPERATUR: air temperature 
    - ABFLUSS: drain
    - ABFLUSS_ROHDATEN: drain forecast (?)
    
    There might be other type of timeseries data; therefore, we will try to extract the name of each type (category), then create a dataframe
    for each of the category. Each dataframe will be stored as a new table in the database.
"""

timeseries_df = spark.createDataFrame(timeseries_data)
timeseries_df = timeseries_df.withColumn("timeseries_data", explode("timeseries_data")).select("*")
timeseries_df = timeseries_df.withColumn("category",element_at(col("timeseries_data"),"longname"))
timeseries_grouped = timeseries_df.groupBy("category").agg(collect_list(struct(col("uuid"), col("timeseries_data"))).alias("timeseries_data"))
timeseries_grouped.show(truncate=True)

# joined_df = df_flattened.join(timeseries_df, df_flattened.uuid == timeseries_df.uuid, 'inner').drop(timeseries_df.uuid)
# joined_df.printSchema()
# joined_df.show(truncate=False)
# print(joined_df.head(5))

+--------------------+--------------------+
|            category|     timeseries_data|
+--------------------+--------------------+
|WASSERSTANDVORHER...|[{e6d68ab7-5c27-4...|
|      LUFTTEMPERATUR|[{e6d68ab7-5c27-4...|
|    WASSERTEMPERATUR|[{e6d68ab7-5c27-4...|
|WASSERSTAND ROHDATEN|[{e6d68ab7-5c27-4...|
|             ABFLUSS|[{616dd98e-816d-4...|
|    ABFLUSS_ROHDATEN|[{7cb7461b-3530-4...|
|WASSERTEMPERATUR ...|[{a6ee8177-107b-4...|
+--------------------+--------------------+



In [9]:
distinct_category = timeseries_grouped.select('category').distinct().rdd.map(lambda row: row[0]).collect()
distinct_category

['WASSERSTANDVORHERSAGE',
 'LUFTTEMPERATUR',
 'WASSERTEMPERATUR',
 'WASSERSTAND ROHDATEN',
 'ABFLUSS',
 'ABFLUSS_ROHDATEN',
 'WASSERTEMPERATUR ROHDATEN']

In [23]:
timeseries_df = {}
for category in distinct_category:
    temp_df = timeseries_grouped.filter(timeseries_grouped["category"] == category) \
                                .select(explode("timeseries_data").alias("timeseries_data")) \
                                .select(col("timeseries_data.uuid"), col("timeseries_data.timeseries_data"))
    timeseries_df[category] = temp_df.select("uuid",explode("timeseries_data")) \
                                             .groupBy("uuid").pivot("key").agg(first("value"))
    
timeseries_df["WASSERSTANDVORHERSAGE"].show()

+--------------------+--------------------+--------------------+------------+--------------------+---------+--------------------+----+
|                uuid|             comment|                 end|equidistance|            longname|shortname|               start|unit|
+--------------------+--------------------+--------------------+------------+--------------------+---------+--------------------+----+
|070b1eb4-3872-4e0...|{longDescription=...|2024-07-16T06:00:...|         120|WASSERSTANDVORHER...|       WV|2024-07-13T16:00:...|  cm|
|094b96e5-caeb-46d...|{longDescription=...|2024-07-16T06:00:...|         120|WASSERSTANDVORHER...|       WV|2024-07-13T16:00:...|  cm|
|13e91b77-90f3-41a...|{longDescription=...|2024-07-18T06:00:...|         120|WASSERSTANDVORHER...|       WV|2024-07-13T16:00:...|  cm|
|1d26e504-7f9e-480...|{longDescription=...|2024-07-16T07:00:...|         120|WASSERSTANDVORHER...|       WV|2024-07-13T17:00:...|  cm|
|1edc5fa4-88af-47f...|{longDescription=...|2024-07-16T0

In [9]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.6 MB/s eta 0:00:0000:0100:01


In [40]:
import psycopg2
from psycopg2.extras import execute_values

def table_exists(table_name, conn_params):
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    cursor.execute("""
        SELECT EXISTS (
            SELECT 1
            FROM information_schema.tables 
            WHERE table_schema = 'public'
            AND table_name = %s
        );
    """, (table_name,))
    exists = cursor.fetchone()[0]
    cursor.close()
    conn.close()
    return exists

In [41]:
def convert_type(spark_type):
    mapping = {
        "bigint": "BIGINT",
        "int": "INTEGER",
        "string": "VARCHAR",
        "double": "DOUBLE PRECISION"
    }
    return mapping.get(spark_type, "TEXT")

In [42]:
def create_table(df, table_name, conn_params):
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    
    columns = ", ".join([f'"{field.name}" {convert_type(field.dataType.simpleString())}' for field in df.schema.fields])
    create_table_sql = f"CREATE TABLE {table_name} ({columns});"
    print(create_table_sql)
    cursor.execute(create_table_sql)
    conn.commit()
    cursor.close()
    conn.close()

In [43]:
def insert_data(table, df, conn_param):
    columns = [f'"{col}"' for col in df.schema.names]
    data = [tuple(row) for row in df.collect()]

    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    insert_query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES %s"
    execute_values(cursor, insert_query, data)
    conn.commit()
    cursor.close()
    conn.close()

In [44]:
conn_params = {
    "dbname": "db",
    "user": "postgres",
    "password": "12345",
    "host": "pg_container",
    "port": 5432
}

for category in distinct_category:
    table_name = category.replace(" ", "_").lower()
    if not table_exists(table_name, conn_params):
        create_table(timeseries_df[category], table_name, conn_params)
    insert_data(table_name, timeseries_df[category], conn_params)

CREATE TABLE wasserstandvorhersage ("uuid" VARCHAR, "comment" VARCHAR, "end" VARCHAR, "equidistance" VARCHAR, "longname" VARCHAR, "shortname" VARCHAR, "start" VARCHAR, "unit" VARCHAR);
CREATE TABLE lufttemperatur ("uuid" VARCHAR, "equidistance" VARCHAR, "longname" VARCHAR, "shortname" VARCHAR, "unit" VARCHAR);
CREATE TABLE wassertemperatur ("uuid" VARCHAR, "comment" VARCHAR, "equidistance" VARCHAR, "longname" VARCHAR, "shortname" VARCHAR, "unit" VARCHAR);
CREATE TABLE wasserstand_rohdaten ("uuid" VARCHAR, "equidistance" VARCHAR, "gaugeZero" VARCHAR, "longname" VARCHAR, "shortname" VARCHAR, "unit" VARCHAR);
CREATE TABLE abfluss ("uuid" VARCHAR, "equidistance" VARCHAR, "longname" VARCHAR, "shortname" VARCHAR, "unit" VARCHAR);
CREATE TABLE abfluss_rohdaten ("uuid" VARCHAR, "equidistance" VARCHAR, "longname" VARCHAR, "shortname" VARCHAR, "unit" VARCHAR);
CREATE TABLE wassertemperatur_rohdaten ("uuid" VARCHAR, "equidistance" VARCHAR, "longname" VARCHAR, "shortname" VARCHAR, "unit" VARCHAR);

In [45]:
conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()
for category in distinct_category:
    cursor.execute("SELECT * FROM lufttemperatur")

In [48]:
conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()
cursor.execute("SELECT * FROM wasserstandvorhersage ")
rows = cursor.fetchall()
for row in rows:
    print(row)

('070b1eb4-3872-4e07-b2e5-e25fd9251b93', '{longDescription=Vorhersagen und Abschätzungen vom: 12.07.2024 um 06:00 Uhr, Quelle: WSA ElbeWeitere Informationen zur Unterscheidung von Vorhersage und Abschätzung finden Sie auf den Seiten der Bundesanstalt für Gewässerkunde., shortDescription=wsa-md}', '2024-07-16T06:00:00+02:00', '120', 'WASSERSTANDVORHERSAGE', 'WV', '2024-07-13T16:00:00+02:00', 'cm')
('094b96e5-caeb-46d3-a8ee-d44182add069', '{longDescription=Vorhersagen und Abschätzungen vom: 12.07.2024 um 06:00 Uhr, Quelle: WSA ElbeWeitere Informationen zur Unterscheidung von Vorhersage und Abschätzung finden Sie auf den Seiten der Bundesanstalt für Gewässerkunde., shortDescription=wsa-md}', '2024-07-16T06:00:00+02:00', '120', 'WASSERSTANDVORHERSAGE', 'WV', '2024-07-13T16:00:00+02:00', 'cm')
('13e91b77-90f3-41a5-a320-641748e9c311', '{longDescription=Vorhersagen und Abschätzungen vom: 12.07.2024 um 06:00 Uhr, Quelle: WSA ElbeWeitere Informationen zur Unterscheidung von Vorhersage und Absch

In [10]:
import psycopg2
from psycopg2.extras import execute_values

conn_params = {
    "dbname": "db",
    "user": "postgres",
    "password": "12345",
    "host": "pg_container",
    "port": 5432
}

conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE station_timeseries (
    agency VARCHAR(255),
    km DOUBLE PRECISION,
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    longname VARCHAR(255),
    number VARCHAR(255),
    shortname VARCHAR(255),
    uuid UUID,
    water_longname VARCHAR(255),
    water_shortname VARCHAR(255),
    category VARCHAR(255),
    timeseries_uuid UUID,
    timeseries_end TIMESTAMPTZ,
    timeseries_unit VARCHAR(50),
    timeseries_start TIMESTAMPTZ,
    timeseries_comment TEXT,
    timeseries_longname VARCHAR(255),
    timeseries_shortname VARCHAR(50),
    timeseries_equidistance VARCHAR(50)
);


""")
conn.commit()

In [11]:
import psycopg2
from psycopg2.extras import execute_values

# Function to insert data into PostgreSQL
def insert_data(table, data, columns):
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    insert_query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES %s"
    execute_values(cursor, insert_query, data)
    conn.commit()
    cursor.close()
    conn.close()

# Convert Spark DataFrame to list of tuples for insertion into PostgreSQL
joined_data = [tuple(row) for row in joined_df.collect()]

# Define the columns
columns = [
    "agency", "km", "latitude", "longitude", "longname", "number", "shortname", 
    "uuid", "water_longname", "water_shortname", "timeseries_uuid", "category", 
    "unit", "start", "end", "comment", "equidistance"
]

# Insert joined data into PostgreSQL
insert_data("station_timeseries", joined_data, columns)

conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()
cursor.execute("SELECT * FROM station_timeseries")
rows = cursor.fetchall()
for row in rows:
    print(row)

cursor.close()
conn.close()


NameError: name 'joined_df' is not defined